In [ ]:
# # %% ---------------------
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader, random_split
# import numpy as np
# from pathlib import Path
# from tqdm import tqdm
# from sklearn.metrics import f1_score
# import matplotlib.pyplot as plt
# import torch.nn.functional as F

# import sys

# # Aggiungi la cartella che contiene "PretrainedSED" al PYTHONPATH
# sys.path.insert(0, "/Users/Q540900/Desktop/Sparkling---Pattern-Classification-Project/04 - Model Training/Bonus")

# from PretrainedSED.models.atstframe.ATSTF_wrapper import ATSTWrapper

# import os
# os.makedirs("resources", exist_ok=True)
# # os.symlink("PretrainedSED/resources/ATST-F_strong_1.pt", "resources/ATST-F_strong_1.pt")



# # %% ---------------------
# SEGMENTS_DIR = Path("data/segments")
# BATCH_SIZE = 32
# EPOCHS = 50
# LEARNING_RATE = 1e-5
# NUM_CLASSES = 10
# DEVICE = torch.device("mps")
# PATIENCE = 5

# CLASSES = ['Speech', 'Dog Bark', 'Rooster Crow', 'Shout',
#            'Lawn Mower', 'Chainsaw', 'Jackhammer',
#            'Power Drill', 'Horn Honk', 'Siren']

# # %% ---------------------
# class SegmentDataset(Dataset):
#     def __init__(self, folder):
#         self.files = list(Path(folder).glob("*.npz"))
#     def __len__(self):
#         return len(self.files)
#     def __getitem__(self, idx):
#         data = np.load(self.files[idx])
#         X = torch.tensor(data["logmel"], dtype=torch.float32).unsqueeze(0)
#         y = torch.tensor(data["labels"], dtype=torch.float32)
#         return X, y

# # %% ---------------------
# dataset = SegmentDataset(SEGMENTS_DIR)
# n = len(dataset)
# n_valid = int(0.2 * n)
# n_train = n - n_valid
# train_ds, valid_ds = random_split(dataset, [n_train, n_valid])
# train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
# valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE)

# import torch.nn as nn

# class ATSTClassifier(nn.Module):
#     def __init__(self, backbone, embed_dim=768, num_classes=10):
#         super().__init__()
#         self.backbone = backbone
#         self.head = nn.Sequential(
#             nn.Linear(embed_dim, num_classes),
#             nn.Sigmoid()
#         )

#     def forward(self, x):  # x shape: [B, 1, T, F]
#         embeddings = self.backbone(x)  # expected shape: [B, T, D]
#         x_cls = embeddings[:, 0, :]    # pick token CLS
#         return self.head(x_cls)



# # %% ---------------------
# from PretrainedSED.models.atstframe.ATSTF_wrapper import ATSTWrapper
# from PretrainedSED.models.prediction_wrapper import PredictionsWrapper

# # Crea backbone (ATST) e wrapper
# atst = ATSTWrapper()
# wrapper = PredictionsWrapper(atst, checkpoint="ATST-F_strong_1", head_type=None)

# # Copia i pesi dal wrapper nel tuo backbone
# atst.load_state_dict(wrapper.model.state_dict(), strict=False)

# # Ora usa il tuo modello con classificatore custom
# model = ATSTClassifier(atst, embed_dim=768, num_classes=NUM_CLASSES)
# # model.load_state_dict(torch.load("best_model.pt", map_location=DEVICE))
# model.to(DEVICE)
# print('modello caricato con successo')

# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2)

# # %% ---------------------
# def mean_average_precision_k(y_true, y_score, k):
#     aps = []
#     for true, score in zip(y_true, y_score):
#         top_k = np.argsort(score)[::-1][:k]
#         true_labels = np.where(true == 1)[0]
#         hits, score_sum = 0, 0.0
#         for i, label in enumerate(top_k):
#             if label in true_labels:
#                 hits += 1
#                 score_sum += hits / (i + 1)
#         aps.append(score_sum / min(len(true_labels), k) if len(true_labels) > 0 else 0.0)
#     return np.mean(aps)

# # %% ---------------------
# best_loss = float('inf')
# early_stop_counter = 0

# train_losses, val_losses = [], []
# f1_micros, map5s, map10s = [], [], []

# print('Comincio il training')
# for epoch in range(1, EPOCHS + 1):
#     model.train()
#     total_tr = 0.
#     for X, y in tqdm(train_loader, desc=f"Epoch {epoch:02d} [Train]", leave=False):
#         X, y = X.to(DEVICE), y.to(DEVICE)
#         logits = model(X)
#         loss = criterion(logits, y)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         total_tr += loss.item() * X.size(0)
#     train_loss = total_tr / n_train
#     train_losses.append(train_loss)

#     # Validation
#     model.eval()
#     total_val = 0.
#     all_targets, all_preds, all_probs = [], [], []
#     with torch.no_grad():
#         for X, y in tqdm(valid_loader, desc=f"Epoch {epoch:02d} [Val]", leave=False):
#             X, y = X.to(DEVICE), y.to(DEVICE)
#             logits = model(X)
#             loss = criterion(logits, y)
#             total_val += loss.item() * X.size(0)
#             probs = torch.sigmoid(logits).cpu().numpy()
#             preds = (probs > 0.5).astype(int)
#             all_targets.append(y.cpu().numpy())
#             all_preds.append(preds)
#             all_probs.append(probs)

#     val_loss = total_val / n_valid
#     val_losses.append(val_loss)

#     y_true = np.vstack(all_targets)
#     y_pred = np.vstack(all_preds)
#     y_score = np.vstack(all_probs)

#     f1_micro = f1_score(y_true, y_pred, average="micro", zero_division=0)
#     map5 = mean_average_precision_k(y_true, y_score, 5)
#     map10 = mean_average_precision_k(y_true, y_score, 10)

#     f1_micros.append(f1_micro)
#     map5s.append(map5)
#     map10s.append(map10)

#     print(f"Epoch {epoch:02d} | train_loss={train_loss:.4f}, val_loss={val_loss:.4f} | F1_micro={f1_micro:.4f} | mAP@5={map5:.4f} | mAP@10={map10:.4f}")

#     if val_loss < best_loss:
#         best_loss = val_loss
#         torch.save(model.state_dict(), "best_model.pt")
#         print("🏆 New best model saved")
#         early_stop_counter = 0
#     else:
#         early_stop_counter += 1
#         if early_stop_counter >= PATIENCE:
#             print("⏹ Early stopping triggered")
#             break

#     scheduler.step(val_loss)

# # %% ---------------------
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 3, 1)
# plt.plot(train_losses, label='Train')
# plt.plot(val_losses, label='Val')
# plt.title("Loss")
# plt.legend()

# plt.subplot(1, 3, 2)
# plt.plot(f1_micros)
# plt.title("F1-micro")

# plt.subplot(1, 3, 3)
# plt.plot(map5s, label="mAP@5")
# plt.plot(map10s, label="mAP@10")
# plt.title("mAP")
# plt.legend()
# plt.tight_layout()
# plt.show()


Loading pretrained checkpoint:  ATST-F_strong_1
modello caricato con successo
Comincio il training


Epoch 01 | train_loss=0.6948, val_loss=0.6933 | F1_micro=0.0084 | mAP@5=0.0081 | mAP@10=0.0105
🏆 New best model saved


Epoch 02 | train_loss=0.6932, val_loss=0.6932 | F1_micro=0.0084 | mAP@5=0.0070 | mAP@10=0.0099
🏆 New best model saved


Epoch 03 | train_loss=0.6932, val_loss=0.6932 | F1_micro=0.0084 | mAP@5=0.0061 | mAP@10=0.0096
🏆 New best model saved


KeyboardInterrupt: 

In [17]:
# ==============================================================
# Fine‑tuning Frame‑MN (AudioSet‑strong ckpt) on MLPC segments
# ==============================================================
#  - 1st stage: freeze backbone, train head (5 epochs)
#  - 2nd stage: fine‑tune whole net with lower LR
#  - Regularisation: dropout head, weight‑decay, SpecAugment
#  - Metrics: val‑loss, F1‑micro, mAP@5, mAP@10
# ==============================================================

##########################
# 1) LIBRARIES & PATHS   #
##########################
import random, numpy as np, matplotlib.pyplot as plt, torch, torch.nn as nn, torch.optim as optim
from pathlib import Path
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import f1_score

from PretrainedSED.models.frame_mn.Frame_MN_wrapper import FrameMNWrapper
from PretrainedSED.models.prediction_wrapper import PredictionsWrapper

SEGMENTS_DIR = Path("data/segments")   # <-- cartella dei .npz
DEVICE       = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda" if torch.cuda.is_available() else "cpu")

MLPC_CLASSES = [
    'Speech','Dog Bark','Rooster Crow','Shout',
    'Lawn Mower','Chainsaw','Jackhammer',
    'Power Drill','Horn Honk','Siren']
NC = len(MLPC_CLASSES)

##########################
# 2) HYPER‑PARAMETERS    #
##########################
BATCH          = 32
EPOCHS         = 40
PATIENCE       = 5
FREEZE_EPOCHS  = 5          # fase 1
LR_HEAD_1      = 1e-3
LR_HEAD_2      = 5e-4
LR_BACKBONE    = 3e-5
WD_HEAD        = 1e-4
WD_BACKBONE    = 1e-2
DROPOUT_P      = 0.5

##########################
# 3) DATASET + AUGMENT   #
##########################
class SegmentDS(torch.utils.data.Dataset):
    """Carica ogni segmento come log‑Mel (T,F) e label multi‑hot."""
    def __init__(self, folder: Path, augment: bool = False):
        self.files   = list(folder.glob("*.npz"))
        self.augment = augment
    def __len__(self): return len(self.files)
    def _specaugment(self, spec: torch.Tensor):
        """Time‑mask & Freq‑mask soft."""
        # spec: [T,F]
        if random.random() < 0.5:
            t0 = random.randint(0, max(0, spec.shape[0]-10)); spec[t0:t0+10] = 0
        if random.random() < 0.5:
            f0 = random.randint(0, max(0, spec.shape[1]-4));  spec[:,f0:f0+4] = 0
        return spec
    def __getitem__(self, idx):
        d     = np.load(self.files[idx])
        spec  = torch.tensor(d['logmel']).float()   # [T,F]
        if self.augment: spec = self._specaugment(spec)
        x     = spec.unsqueeze(0)                   # [1,T,F]
        y     = torch.tensor(d['labels']).float()
        return x, y

# split train / val
full_ds        = SegmentDS(SEGMENTS_DIR)
val_len        = int(0.2*len(full_ds))
train_ds, val_ds = random_split(full_ds, [len(full_ds)-val_len, val_len], generator=torch.Generator().manual_seed(42))
train_ds.dataset.augment = True  # solo train

dl_tr  = DataLoader(train_ds, BATCH, shuffle=True)
dl_val = DataLoader(val_ds,   BATCH)

##########################
# 4) CLASS‑IMBALANCE     #
##########################
#   pos_weight = (1‑p)/p  per BCE
pos_cnt = torch.zeros(NC)
for _, y in DataLoader(train_ds, BATCH):
    pos_cnt += y.sum(0)
pos_rate   = pos_cnt / len(train_ds)
pos_weight = ((1.0 - pos_rate) / (pos_rate + 1e-6)).to(DEVICE)
criterion  = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

##########################
# 5) BACKBONE + HEAD     #
##########################
width   = 1.0                         # 0.6 → frame_mn06; 1.0 → frame_mn10
backbone = FrameMNWrapper(width)
embed_d  = backbone.state_dict()['frame_mn.features.16.1.bias'].shape[0]

# carica ckpt AudioSet‑strong (+stripping head)
wrapper = PredictionsWrapper(
    backbone,
    checkpoint=f"frame_mn{int(width*10):02d}_strong_1",
    head_type=None)
backbone.load_state_dict(wrapper.model.state_dict(), strict=False)

# nuova testa
class Head(nn.Sequential):
    def __init__(self, in_d, n_cls, p=DROPOUT_P):
        super().__init__(nn.Dropout(p), nn.Linear(in_d, n_cls))
head = Head(embed_d, NC, DROPOUT_P)

class Net(nn.Module):
    def __init__(self, backbone, head):
        super().__init__()
        self.backbone, self.head = backbone, head
    def forward(self, x):
        z = self.backbone(x)      # [B,250,D]
        return self.head(z[:,0])  # token CLS

net = Net(backbone, head).to(DEVICE)

##########################
# 6) OPTIMIZER & SCHED   #
##########################
# fase 1: freeze backbone
for p in net.backbone.parameters():
    p.requires_grad = False
optimizer = optim.AdamW(net.head.parameters(), lr=LR_HEAD_1, weight_decay=WD_HEAD)

##########################
# 7) mAP@k utility        #
##########################
def map_k(y_true, y_score, k=5):
    """Mean Average Precision @k (rows = samples)."""
    aps = []
    for t, s in zip(y_true, y_score):
        topk = np.argsort(s)[::-1][:k]
        rel  = set(np.where(t==1)[0])
        hit = 0; score = 0.0
        for i, lab in enumerate(topk, 1):
            if lab in rel:
                hit += 1; score += hit / i
        aps.append(score / min(len(rel), k) if rel else 0.0)
    return np.mean(aps)

##########################
# 8) TRAIN / VALID LOOP  #
##########################
H = {k:[] for k in ('tr','val','f1','m5','m10')}
best_loss, bad = 1e9, 0
for ep in range(1, EPOCHS+1):
    # ===== train =====
    net.train(); tr_loss = 0
    for x,y in tqdm(dl_tr, desc=f"Ep{ep:02d}[tr]", leave=False):
        x,y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(net(x), y)
        loss.backward(); optimizer.step()
        tr_loss += loss.item()*x.size(0)
    tr_loss /= len(train_ds)

    # ===== validation =====
    net.eval(); val_loss = 0; preds, targets = [], []
    with torch.no_grad():
        for x,y in tqdm(dl_val, desc=f"Ep{ep:02d}[val]", leave=False):
            x,y = x.to(DEVICE), y.to(DEVICE)
            logit = net(x)
            val_loss += criterion(logit, y).item()*x.size(0)
            preds.append(torch.sigmoid(logit).cpu()); targets.append(y.cpu())
    val_loss /= len(val_ds)
    P = torch.vstack(preds).numpy(); T = torch.vstack(targets).numpy()
    f1  = f1_score(T, (P>.5).astype(int), average='micro', zero_division=0)
    m5  = map_k(T,P,5); m10 = map_k(T,P,10)

    # log
    H['tr'].append(tr_loss); H['val'].append(val_loss)
    H['f1'].append(f1); H['m5'].append(m5); H['m10'].append(m10)
    print(f"Ep{ep:02d} tr={tr_loss:.3f} val={val_loss:.3f} F1={f1:.3f} mAP5={m5:.3f} mAP10={m10:.3f}")

    # early‑stop & checkpoint
    if val_loss < best_loss:
        best_loss, bad = val_loss, 0
        torch.save(net.state_dict(), "best_frame_mn.pt")
        print("   🔥 checkpoint!")
    else:
        bad += 1
        if bad == PATIENCE:
            print("   ⏹️  Early‑stop"); break

    # fine‑tune backbone dopo FREEZE_EPOCHS
    if ep == FREEZE_EPOCHS:
        for p in net.backbone.parameters():
            p.requires_grad = True
        optimizer = optim.AdamW([
            {"params": net.head.parameters(),     "lr": LR_HEAD_2, "weight_decay": WD_HEAD},
            {"params": net.backbone.parameters(), "lr": LR_BACKBONE, "weight_decay": WD_BACKBONE}
        ])

##########################
# 9) PLOT                #
##########################
plt.figure(figsize=(13,4))
plt.subplot(1,3,1); plt.plot(H['tr'],label='train'); plt.plot(H['val'],label='val'); plt.legend(); plt.title('Loss')
plt.subplot(1,3,2); plt.plot(H['f1']); plt.title('F1‑micro')
plt.subplot(1,3,3); plt.plot(H['m5'],label='mAP5'); plt.plot(H['m10'],label='mAP10'); plt.legend(); plt.title('mAP')
plt.tight_layout(); plt.show()


dilation_list_t_dim: 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
MN(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): C

Ep01[tr]:   0%|          | 0/2969 [00:00<?, ?it/s]

Ep01[val]:   0%|          | 0/743 [00:00<?, ?it/s]

Ep01 tr=1.572 val=1.376 F1=0.014 mAP5=0.014 mAP10=0.015
   🔥 checkpoint!


Ep02[tr]:   0%|          | 0/2969 [00:00<?, ?it/s]

...

Ep06[val]:   0%|          | 0/743 [00:00<?, ?it/s]

Ep06 tr=1.620 val=1.421 F1=0.016 mAP5=0.015 mAP10=0.016


Ep07[tr]:   0%|          | 0/2969 [00:00<?, ?it/s]

Ep07[val]:   0%|          | 0/743 [00:00<?, ?it/s]

KeyboardInterrupt: 